### Carregando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.linear_model import LogisticRegression

### Importanto e tratando os datasets de cada pais

In [2]:
CAvideos = pd.read_csv('CAvideos.csv')
DEvideos = pd.read_csv('DEvideos.csv')
FRvideos = pd.read_csv('FRvideos.csv')
GBvideos = pd.read_csv('GBvideos.csv')
USvideos = pd.read_csv('USvideos.csv')

In [3]:
print("CA DataSet shape: "+str(CAvideos.shape))
print("DE DataSet shape: "+str(DEvideos.shape))
print("FR DataSet shape: "+str(FRvideos.shape))
print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

CA DataSet shape: (40881, 16)
DE DataSet shape: (40840, 16)
FR DataSet shape: (40724, 16)
GB DataSet shape: (38916, 16)
US DataSet shape: (40949, 16)


In [4]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64



In [5]:
CAvideos = CAvideos.dropna(axis=0)
DEvideos = DEvideos.dropna(axis=0)
FRvideos = FRvideos.dropna(axis=0)
GBvideos = GBvideos.dropna(axis=0)
USvideos = USvideos.dropna(axis=0)

In [6]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64



In [7]:
#print("CA DataSet shape: "+str(CAvideos.shape))
#print("DE DataSet shape: "+str(DEvideos.shape))
#print("FR DataSet shape: "+str(FRvideos.shape))
#print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

US DataSet shape: (40379, 16)


## Correção de formato de data-hora para padrão Unix

In [8]:
USvideos['trending_date'] = pd.to_datetime(USvideos['trending_date'], format='%y.%d.%m')
USvideos['publish_time'] = pd.to_datetime(USvideos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

USvideos.insert(4, 'publish_date', USvideos['publish_time'].dt.date)
USvideos['publish_time'] = USvideos['publish_time'].dt.time
USvideos['publish_date']=pd.to_datetime(USvideos['publish_date'])

In [9]:
USvideos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [10]:
USvideos_lastentry = USvideos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr..."
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...


In [11]:
print("US DataSet shape: "+str(USvideos_lastentry.shape))

US DataSet shape: (6254, 17)


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [12]:
with open('US_category_id.json') as f:
    UScategory = json.load(f)

In [13]:
def category_replace(c_id):
    for i in UScategory["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [14]:
USvideos_lastentry['category_name'] = USvideos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)

In [15]:
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music


## Normalização das features numéricas

In [16]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6.254000e+03,6.254000e+03,6.254000e+03,6.254000e+03
mean,20.408539,1.975222e+06,5.592469e+04,2.828081e+03,6.289861e+03
std,7.256811,7.092548e+06,1.942315e+05,2.433277e+04,2.943713e+04
min,1.000000,5.590000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.648875e+05,2.931000e+03,1.300000e+02,3.870000e+02
50%,24.000000,5.270375e+05,1.222400e+04,4.485000e+02,1.287500e+03
75%,25.000000,1.490734e+06,3.898500e+04,1.516000e+03,4.110750e+03
max,43.000000,2.252119e+08,5.613827e+06,1.643059e+06,1.228655e+06


In [17]:
# Normalização
USvideos_lastentry['views'] = (USvideos_lastentry['views']-USvideos_lastentry['views'].min())/(USvideos_lastentry['views'].max()-USvideos_lastentry['views'].min())
USvideos_lastentry['likes'] = (USvideos_lastentry['likes']-USvideos_lastentry['likes'].min())/(USvideos_lastentry['likes'].max()-USvideos_lastentry['likes'].min())
USvideos_lastentry['dislikes'] = (USvideos_lastentry['dislikes']-USvideos_lastentry['dislikes'].min())/(USvideos_lastentry['dislikes'].max()-USvideos_lastentry['dislikes'].min())
USvideos_lastentry['comment_count'] = (USvideos_lastentry['comment_count']-USvideos_lastentry['comment_count'].min())/(USvideos_lastentry['comment_count'].max()-USvideos_lastentry['comment_count'].min())


In [18]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6254.000000,6254.000000,6254.000000,6254.000000
mean,20.408539,0.008768,0.009962,0.001721,0.005119
std,7.256811,0.031493,0.034599,0.014809,0.023959
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,0.000730,0.000522,0.000079,0.000315
50%,24.000000,0.002338,0.002177,0.000273,0.001048
75%,25.000000,0.006617,0.006944,0.000923,0.003346
max,43.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
#positive sentiment = 60% da quantidade de likes+dislikes
USvideos_lastentry['sentiment'] = (USvideos_lastentry['likes']-USvideos_lastentry['dislikes']) >= 0.6*(USvideos_lastentry['likes']+USvideos_lastentry['dislikes']) 
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",0.045760,0.063607,0.129622,0.117848,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming,False
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",0.003547,0.002866,0.000397,0.001262,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics,True
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",0.004456,0.008573,0.000541,0.004463,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation,True
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",0.004010,0.002036,0.001403,0.002038,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style,False
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",0.007172,0.016460,0.001130,0.003721,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music,True


In [20]:
train_data = USvideos_lastentry.sample(frac=0.8, random_state=200)
test_data  = USvideos_lastentry.drop(train_data.index)
print(train_data.shape, test_data.shape)

(5003, 19) (1251, 19)


## Bag of words Title

In [21]:
count_vectorizer = CountVectorizer()
x_train = count_vectorizer.fit_transform(train_data['title'].values)
y_train = train_data['sentiment'].values

In [22]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
x_test = count_vectorizer.transform(test_data['title'].values)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [24]:
probs = classifier.predict_proba(x_test)
print(probs)

[[0.16959882 0.83040118]
 [0.15791647 0.84208353]
 [0.06448402 0.93551598]
 ...
 [0.13847831 0.86152169]
 [0.60941015 0.39058985]
 [0.58632081 0.41367919]]


In [25]:
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.7561950439648282
0.7511947763267574


In [26]:
probs = classifier.predict_proba(count_vectorizer.transform(USvideos_lastentry['title']))

In [27]:
USvideos_title = USvideos_lastentry

In [28]:
USvideos_title['predicted_sentiment'] = probs.max(axis=1)

In [29]:
USvideos_title = USvideos_title.sort_values('predicted_sentiment', ascending=False)
USvideos_title.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
16340,exsOim0Lyl4,2018-02-05,President Donald Trump’s State of the Union Ad...,NBC News,2018-01-31,25,06:36:37,"nbc news|""breaking news""|""us news""|""world news...",0.007988,0.003992,0.010896,0.004686,https://i.ytimg.com/vi/exsOim0Lyl4/default.jpg,False,False,False,Watch President Donald Trump's 2018 State of t...,News & Politics,False,0.998954
2780,YcCrw6d3UX4,2017-11-27,President Donald Trump Pardons His First Thank...,NBC News,2017-11-21,25,19:18:02,"nbc news|""nbc""|""news""|""news channel""|""news sta...",0.000025,0.000010,0.000018,0.000044,https://i.ytimg.com/vi/YcCrw6d3UX4/default.jpg,False,False,False,President Donald Trump led the time-honored tr...,News & Politics,False,0.998461
15703,HUZRjGEwqCg,2018-02-02,State of the Union 2018 live stream: President...,ABC News,2018-01-31,25,05:05:25,"donald trump|""donald trump state of the union""...",0.000992,0.000481,0.001032,0.000978,https://i.ytimg.com/vi/HUZRjGEwqCg/default.jpg,False,False,False,"The president discusses healthcare, immigratio...",News & Politics,False,0.997494
30142,4Qs8herYTqY,2018-04-21,Professional Makeup Artist Watching Me Do my M...,Nyma Tang,2018-04-04,26,00:27:14,"Nyma Tang|""the darkest shade""|""contour for dar...",0.002787,0.006468,0.000187,0.002443,https://i.ytimg.com/vi/4Qs8herYTqY/default.jpg,False,False,False,"Hey guys, \n\nCome hang out with me and Daniel...",Howto & Style,True,0.997085
2980,EbuCLC_wVlM,2017-11-28,Victoria’s Secret Model Sara Sampaio’s Easy Bo...,Vogue,2017-11-21,24,14:54:23,"beauty|""tutorial""|""how-to""|""how to""|""tips""|""tr...",0.001984,0.002998,0.000111,0.000438,https://i.ytimg.com/vi/EbuCLC_wVlM/default.jpg,False,False,False,For the 26-year-old model from Portugal and Vi...,Entertainment,True,0.996605


## Bag of words Descrição

In [30]:
count_vectorizer = CountVectorizer()
x_train = count_vectorizer.fit_transform(train_data['description'].values)
y_train = train_data['sentiment'].values

In [31]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
x_test = count_vectorizer.transform(test_data['description'].values)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [33]:
probs = classifier.predict_proba(x_test)
print(probs)

[[5.46977275e-02 9.45302273e-01]
 [2.82682880e-04 9.99717317e-01]
 [1.46243121e-07 9.99999854e-01]
 ...
 [5.24426630e-02 9.47557337e-01]
 [2.63551361e-01 7.36448639e-01]
 [9.14761383e-01 8.52386173e-02]]


In [34]:
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.7609912070343725
0.7538992312679448


In [35]:
probs = classifier.predict_proba(count_vectorizer.transform(USvideos_lastentry['description']))

In [36]:
USvideos_description = USvideos_lastentry

In [37]:
USvideos_description['predicted_sentiment'] = probs.max(axis=1)

In [38]:
USvideos_description = USvideos_description.sort_values('predicted_sentiment', ascending=False)
USvideos_description.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
13971,sTnCS91HgcE,2018-01-24,GET READY WITH ME | Chit Chat with Winnie Harlow,LaToya Forever,2018-01-18,24,23:03:24,"latoyaforever|""chit chat makeup""|""grwm""|""chit ...",0.000246,0.000672,0.000019,0.000341,https://i.ytimg.com/vi/sTnCS91HgcE/default.jpg,False,False,False,CHIT CHAT GRWM WITH WINNIE HARLOW\nGot my girl...,Entertainment,True,1.0
12794,YXTzMOmmEfE,2018-01-18,Hayley Kiyoko - Curious [Official Video],Hayley Kiyoko,2018-01-11,10,21:28:39,"Curious|""Expectations""|""Hayley Kiyoko""|""Sleep ...",0.011110,0.027820,0.001043,0.007602,https://i.ytimg.com/vi/YXTzMOmmEfE/default.jpg,False,False,False,Curious Available Now! Download or stream at: ...,Music,True,1.0
5985,xlQLyAOQXjQ,2017-12-13,The FULL COLLECTION of 7-ELEVEN MAKEUP TESTED ...,Tati,2017-12-06,26,18:00:11,"YouTube|""Beauty""|""Makeup""|""Tutorial""|""Review""|...",0.004664,0.006822,0.000817,0.004185,https://i.ytimg.com/vi/xlQLyAOQXjQ/default.jpg,False,False,False,MOST REQUESTED VIDEO EVER!!! 7 ELEVEN launched...,Howto & Style,True,1.0
28487,WOAvDQNNaCA,2018-04-07,MØ - Nostalgia (Lyric Video),MOMOMOYOUTHVEVO,2018-03-29,10,04:00:02,"Nostalgia Mø|""Nostalgia Song""|""Mø""|""Nostalgia ...",0.001425,0.002619,0.000128,0.000496,https://i.ytimg.com/vi/WOAvDQNNaCA/default.jpg,False,False,False,MØ 'Nostalgia' Lyric Video \n\nSubscribe to th...,Music,True,1.0
1678,FrK2j4cPxSc,2017-11-22,Trump Breaks Silence On Roy Moore And Offers D...,Business Insider,2017-11-21,25,21:56:06,"Business Insider|""Donald Trump""|""White House""|...",0.000015,0.000006,0.000009,0.000074,https://i.ytimg.com/vi/FrK2j4cPxSc/default.jpg,False,False,False,President Donald Trump implicitly endorsed emb...,News & Politics,False,1.0


## Bag of words Tag

In [39]:
count_vectorizer = CountVectorizer()
x_train = count_vectorizer.fit_transform(train_data['tags'].values)
y_train = train_data['sentiment'].values

In [40]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
x_test = count_vectorizer.transform(test_data['tags'].values)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [42]:
probs = classifier.predict_proba(x_test)
print(probs)

[[9.08609944e-02 9.09139006e-01]
 [8.89135559e-02 9.11086444e-01]
 [1.82902745e-03 9.98170973e-01]
 ...
 [9.56765235e-02 9.04323476e-01]
 [9.99995233e-01 4.76729747e-06]
 [8.01298028e-01 1.98701972e-01]]


In [43]:
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.7529976019184652
0.7482032045938686


In [44]:
probs = classifier.predict_proba(count_vectorizer.transform(USvideos_lastentry['tags']))

In [45]:
USvideos_tags = USvideos_lastentry

In [46]:
USvideos_tags['predicted_sentiment'] = probs.max(axis=1)

In [47]:
USvideos_tags = USvideos_tags.sort_values('predicted_sentiment', ascending=False)
USvideos_tags.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
20750,Q6vfaPuftmE,2018-02-27,Donald Trump Jr. Says India's Poorest Have Som...,NBC News,2018-02-20,25,16:55:34,"NBC|""NBC News""|""Breaking News""|""US News""|""Worl...",0.000150,0.000026,0.000129,0.000226,https://i.ytimg.com/vi/Q6vfaPuftmE/default.jpg,False,False,False,At the start of a trip to attract buyers for l...,News & Politics,False,1.0
15359,HvPbH30KWLE,2018-01-31,BTS Takes on L.A. | Vogue,Vogue,2018-01-25,10,14:00:31,"bts|""korean band""|""bts band""|""bts band members...",0.005858,0.039839,0.000517,0.009653,https://i.ytimg.com/vi/HvPbH30KWLE/default.jpg,False,False,False,"Last November, BTS cemented their status as a ...",Music,True,1.0
15703,HUZRjGEwqCg,2018-02-02,State of the Union 2018 live stream: President...,ABC News,2018-01-31,25,05:05:25,"donald trump|""donald trump state of the union""...",0.000992,0.000481,0.001032,0.000978,https://i.ytimg.com/vi/HUZRjGEwqCg/default.jpg,False,False,False,"The president discusses healthcare, immigratio...",News & Politics,False,1.0
37791,9sRQQRiltrE,2018-05-30,BTS Get Scared by a Fangirl,TheEllenShow,2018-05-25,24,13:00:00,"ellen|""ellen degeneres""|""the ellen show""|""seas...",0.017784,0.034584,0.002710,0.017625,https://i.ytimg.com/vi/9sRQQRiltrE/default.jpg,False,False,False,K-pop superstars BTS returned to Ellen to talk...,Entertainment,True,1.0
16340,exsOim0Lyl4,2018-02-05,President Donald Trump’s State of the Union Ad...,NBC News,2018-01-31,25,06:36:37,"nbc news|""breaking news""|""us news""|""world news...",0.007988,0.003992,0.010896,0.004686,https://i.ytimg.com/vi/exsOim0Lyl4/default.jpg,False,False,False,Watch President Donald Trump's 2018 State of t...,News & Politics,False,1.0
